In [22]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import pandas as pd
import math
import numpy as np
import os
import time 

import cntk as C
import cntk.tests.test_utils
import pickle
import random
from cntk import sequence
from cntk import load_model
from cntk.device import try_set_default_device, gpu,cpu
from scipy.sparse import csr_matrix

cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components
try_set_default_device(gpu(0))


vocab_size = 80000
num_labels = 19
title_size = 52000
body_size  = 210000
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 300
hidden_dim = 200

max_length_title = 30
max_length_body  = 100



suffix = "180days_sample"
prefix = "/home/t-haohu/IndustryClassifier/Data/"

#data_token_body        = "{}/middle/{}_token_body.txt".format(prefix,suffix)
data_train_sample = "{}/middle/train_{}.txt".format(prefix,suffix)
data_test_sample  = "{}/middle/test_{}.txt".format(prefix,suffix)

data_industry_sample = "{}/ready/industry_{}.wl".format(prefix,suffix)
data_title_sample    = "{}/ready/title_{}.wl".format(prefix,suffix)
data_body_sample     = "{}/ready/body_{}.wl".format(prefix,suffix)

 

def load_data(input_file,title_dict,industry_dict):
    data = open(input_file, encoding = "utf-8").readlines()
    
    data_title = np.zeros((len(data),max_length_title),dtype = np.float32)
    data_label = np.zeros((len(data),1),dtype = np.float32)
    
    
    for index,line in enumerate(data):
        row = line.strip("\n").split("\t")
        
        title    =  row[0]
        #body     =  row[1]
        industry =  row[1]
        
        for jndex,token in enumerate(title.split(" ")):
            if jndex>=max_length_title:
                break
            data_title[index,jndex]=title_dict.get(token,len(title_dict)-1)    
        data_label[index] = industry_dict.get(industry,len(industry_dict))
    return data_title,data_label

def create_model_cnn(embed = False):
    #version 2 : 1 dense layer version3: sigmoid activation in dense
    #
    


    if embed:
        h1= C.layers.Embedding(weights=embedding_title,name='embed_1')(input_xt_one_hot)#
    else:
        h1= C.layers.Embedding(300,name='embed_2')(input_xt_one_hot)#init=embedding,

    filter_num=100

    h2_1=C.layers.Convolution((1,emb_dim),num_filters=filter_num,reduction_rank=0,activation=C.relu)(h1)
    h2_2=C.layers.Convolution((2,emb_dim),num_filters=filter_num,reduction_rank=0,activation=C.relu)(h1)
    h2_3=C.layers.Convolution((3,emb_dim),num_filters=filter_num,reduction_rank=0,activation=C.relu)(h1)

    h3_1=C.layers.MaxPooling((max_length_title-0,1),name='pooling_1')(h2_1)
    h3_2=C.layers.MaxPooling((max_length_title-1,1),name='pooling_2')(h2_2)
    h3_3=C.layers.MaxPooling((max_length_title-2,1),name='pooling_3')(h2_3)
    #h2=BiRecurrence(C.layers.LSTM(hidden_dim), C.layers.LSTM(hidden_dim))(h1)
    h3=C.splice(h3_2,h3_1,h3_3,axis=0)
    drop1 = C.layers.Dropout(0.1)(h3)
    h4=C.layers.Dense(num_labels,name='hidden')(drop1)

    return h4

def batch_iter(data,batch_size, num_epochs, shuffle=True):
    # Generates a batch iterator for a dataset.
    data_size = len(data)
    num_batches_per_epoch = int((data_size-1)/batch_size) + 1
    print('data_size: ', data_size, 'batch_size: ', batch_size, 'num_batches_per_epoch: ', num_batches_per_epoch)
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            random.shuffle(data)
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield data[start_index:end_index]
            

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_sample).readlines()])}
industry_dict =  { x:i for i,x in enumerate([x.strip("\n") for x in open(data_industry_sample).readlines()])}




input_xt = C.input_variable(shape=(max_length_title),  dtype=np.float32)
input_y  = C.input_variable(shape=(1)               ,  dtype=np.int)

input_xt_one_hot = C.one_hot(input_xt, num_classes=len(title_dict)   ,  sparse_output=True)
input_y_one_hot = C.one_hot(input_y  , num_classes=len(industry_dict) ,  sparse_output=True)


test_data  = load_data(data_test_sample,title_dict,industry_dict)
train_data = load_data(data_train_sample,title_dict,industry_dict)

def test(batch_size,model,data):
    scores = model(input_xt)
    predict = C.argmax(scores,axis = 0)
    confuse = np.zeros((num_labels,num_labels))

    test_data_title,test_data_label = data
    batches = batch_iter(list(zip(test_data_title,test_data_label)), batch_size, 1)
    
    for batch in batches:
        batch_data_title,batch_data_label = zip(*batch) 
        output = np.array(predict.eval({input_xt: np.array(batch_data_title)}),dtype=np.int)
        gt = np.array(batch_data_label,dtype=np.int)
        confuse+=fast_hist(output,gt,num_labels)
        
    precision=np.diag(confuse)/np.sum(confuse,axis=0)
    recall = np.diag(confuse)/np.sum(confuse,axis=1)
    accuarcy = np.diag(confuse).sum() / confuse.sum()
    aver_precision=np.nanmean(precision)
    aver_recall = np.nanmean(recall)
   
    print("Precision:{} Recall:{} Acc:{}".format(aver_precision,aver_recall,accuarcy))
    
def train(train_data,num_epochs,learning_rate,batch_size,l2_weight=0):
    
    #learning_rate *= batch_size
    model = create_model_cnn()
    print(C.logging.get_node_outputs(model))
    scores = model(input_xt)

    loss =C.reduce_mean(C.losses.cross_entropy_with_softmax(scores, input_y_one_hot))
    
    # Training
    lr_schedule = C.learning_parameter_schedule(learning_rate)
    #learner = C.adam(scores.parameters, lr=lr_schedule, momentum=0.9,l2_regularization_weight=0)
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=num_epochs)
    momentums = C.momentum_schedule(0.99, minibatch_size=batch_size)
    learner = C.adam(parameters=scores.parameters,#model.parameters,
                     lr=lr_schedule,
                     momentum=momentums,
                     gradient_clipping_threshold_per_sample=15,
                     gradient_clipping_with_truncation=True,
                     l2_regularization_weight=l2_weight)
    trainer = C.Trainer(scores, (loss), [learner], progress_printer)
    
    train_data_title,train_data_label = train_data
    batches = batch_iter(list(zip(train_data_title,train_data_label)), batch_size, num_epochs)

    # training loop
    count = 0
    t = time.time()
    for batch in batches:
        count += 1
        batch_data_title,batch_data_label = zip(*batch)
        
        trainer.train_minibatch({input_xt: np.array(batch_data_title), input_y: np.array(batch_data_label)})
        if count%1398== 0:
            print(count,time.time()-t)
            t=time.time()
            test(batch_size,model,test_data)

    # save model
    model.save('./model/IndustryClassifyCNN.dnn')


In [23]:
train(train_data,num_epochs=20,learning_rate=[5e-4*30]*2+[1e-4*30],batch_size = 30)

[Output('hidden', [#], [19]), Output('Block13273_Output_0', [#], [300 x 1 x 1]), Output('Splice13253_Output_0', [#], [300 x 1 x 1]), Output('pooling_2', [#], [100 x 1 x 1]), Output('Block13044_Output_0', [#], [100 x 29 x 1]), Output('embed_2', [#], [30 x 300]), Output('OneHotOp12907_Output_0', [#], [30 x 23184]), Output('pooling_1', [#], [100 x 1 x 1]), Output('Block12981_Output_0', [#], [100 x 30 x 1]), Output('pooling_3', [#], [100 x 1 x 1]), Output('Block13107_Output_0', [#], [100 x 28 x 1])]
data_size:  41940 batch_size:  30 num_batches_per_epoch:  1398
Learning rate per minibatch: 0.015
1398 18.27876329421997
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600


/home/t-haohu/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/ipykernel_launcher.py:147: RuntimeWarning: invalid value encountered in true_divide


Precision:0.5110806497883653 Recall:0.6923817686419086 Acc:0.6890681502086231
2796 18.42057991027832
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6082792499779451 Recall:0.7130376981205395 Acc:0.7360222531293463
4194 17.433727502822876
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6198075709116803 Recall:0.6513190393174622 Acc:0.7281223922114047
5592 17.409441471099854
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6129659700487289 Recall:0.6548085207460945 Acc:0.7156606397774687
6990 17.552210807800293
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6105506616896933 Recall:0.6341483941328568 Acc:0.7164951321279555
8388 17.567949771881104
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.6027486388223229 Recall:0.6320560028956943 Acc:0.7081502086230876
9786 17.82224178314209
data_size:  17975 batch_size:  30 num_batches_per_epoch:  600
Precision:0.609110